In [122]:
from pymongo import MongoClient
from pprint import pprint

client = MongoClient("localhost", 27017)
client.drop_database("blog")
client.list_database_names()


['admin', 'config', 'local']

In [123]:
db = client.blog
db.list_collection_names()


[]

In [124]:
db.create_collection(
    name="posts",
    validator={
        "$jsonSchema": {
            "bsonType": "object",
            "required": ["title", "text", "creator", "comments"],
            "properties": {
                "title": {
                    "bsonType": "string",
                    "description": "must be a string and is required",
                },
                "text": {
                    "bsonType": "string",
                    "description": "must be a string and is required",
                },
                "creator": {
                    "bsonType": "objectId",
                    "description": "must be an objectid and is required",
                },
                "comments": {
                    "bsonType": "array",
                    "description": "must be an array and is required",
                    "items": {
                        "bsonType": "object",
                        "required": ["text", "author"],
                        "properties": {
                            "text": {
                                "bsonType": "string",
                                "description": "must be a string and is required",
                            },
                            "author": {
                                "bsonType": "objectId",
                                "description": "must be an objectid and is required",
                            },
                        },
                    },
                },
            },
        }
    },
    validationAction="error",
)


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'blog'), 'posts')

In [125]:
users = db.users.insert_many(
    [
        {"name": "Max Schwarzmulller", "age": 29, "email": "max@test.com"},
        {"name": "Manuel Lorenz", "age": 30, "email": "manu@test.com"},
    ]
).inserted_ids
for x in db.users.find():
    pprint(x)


{'_id': ObjectId('645005be562a2565b6cab66e'),
 'age': 29,
 'email': 'max@test.com',
 'name': 'Max Schwarzmulller'}
{'_id': ObjectId('645005be562a2565b6cab66f'),
 'age': 30,
 'email': 'manu@test.com',
 'name': 'Manuel Lorenz'}


In [126]:
try:
    db.posts.insert_one(
        {
            "title": "My first post!",
            "text": "This is my first post.",
            "tags": ["new", "tech"],
            "creator": 1,
            "comments": [{"text": "I like this post.", "author": users[1]}],
        }
    )
except Exception as e:
    pprint(e.args)


("Document failed validation, full error: {'index': 0, 'code': 121, 'errmsg': "
 "'Document failed validation', 'errInfo': {'failingDocumentId': "
 "ObjectId('645005be562a2565b6cab670'), 'details': {'operatorName': "
 "'$jsonSchema', 'schemaRulesNotSatisfied': [{'operatorName': 'properties', "
 "'propertiesNotSatisfied': [{'propertyName': 'creator', 'description': 'must "
 "be an objectid and is required', 'details': [{'operatorName': 'bsonType', "
 "'specifiedAs': {'bsonType': 'objectId'}, 'reason': 'type did not match', "
 "'consideredValue': 1, 'consideredType': 'int'}]}]}]}}}",)


In [127]:
client.close()
